In [1]:
from datetime import datetime

import pandas as pd
from utils import get_location_details

Laminou, Ouèssè, Collines, Bénin


In [2]:
def process_data(data):
    if "m2" in data:
        return int(data.replace("m2", ""))
    elif "Ha" in data:
        return int(data.replace("Ha", "")) * 10000

    else:
        return data

In [3]:
def convert_to_int(value):
    # Try to convert the value to an integer
    try:
        return int(value)
    except ValueError:
        # If a ValueError is raised, remove non-numeric characters and try again
        return int("".join(filter(str.isdigit, str(value))) or 0)

In [4]:
def date_correction(date):
    return date - 2 * (date - datetime.now())

In [5]:
# Read the CSV file
df = pd.read_csv("../terrain_coinafrique.csv", skiprows=1)

In [19]:
df

,title,description,price,area,date,city
0,Parcelle,Détails du produitParcelle size à glo dans la ...,25000000,3600,2024-02,glo
1,Terrains 350m²,Détails du produitBonjour. notre cabinet met e...,12000000,350,2024-02,abomey-calavi
2,Parcelle 325 m2,Détails du produitNous mettons en vente a akpa...,25000000,325,2024-02,akpakpa avotrou
3,Parcelles 631m²,Détails du produitNous mettons en vente pour u...,45000000,684,2024-02,sagla
5,Parcelles,Détails du produitIl est mis en vente à hêvié ...,3200000,500,2024-02,hevié
...,...,...,...,...,...,...
8881,Vente Terrain 450m²,Détails du produitVente d’un ensemble de 10 pa...,2699000,450,2021-02,hevié centre
8882,Vente parcelle 500m²,Détails du produitVente de parcelles de 500m² ...,890000,500,2021-02,zinvié zoumey
8883,Vente parcelles 350 m²,Détails du produitVente d’un ensemble de 22 pa...,399000,350,2021-02,kpomassè
8887,Vente Terrain 501 m²,Détails du produitParcelle de superficie: 501m...,50000000,501,2021-02,cotonou


In [7]:
# Remove duplicate rows
df = df.drop_duplicates()
# Set the column names
df.columns = ["title", "description", "price", "area", "date"]

In [8]:
# Delete  "m2" on column "area" and
df["area"] = df["area"].apply(process_data)
# Convert the "area" column to integers
df["area"] = df["area"].apply(convert_to_int)
df["price"] = df["price"].apply(convert_to_int)

In [9]:
# Delete the column with price < 10000 or area < 100
df = df[df["price"] > 10000]
df = df[df["area"] > 100]

In [10]:
df["date"] = pd.to_datetime(df["date"])
df["date"] = df["date"].apply(date_correction)

In [11]:
# reduce date to years and months
df["date"] = df["date"].dt.strftime("%Y-%m")
# Remove duplicate rows
df = df.drop_duplicates()

In [12]:
# Transform title column on two columns
df[["title", "city"]] = df["title"].str.split("-", n=1, expand=True)
# Delete the column with city null
df = df[df["city"].notna()]

In [13]:
# suppress spaces at the beginning of the city column
df["city"] = df["city"].str.strip()

In [16]:
# transform the city column to lowercase
df["city"] = df["city"].str.lower()

In [23]:
# Get the location details and save on the 4 new column
df[["location"]] = df["city"].apply(get_location_details)

GeocoderInsufficientPrivileges: Non-successful status code 403

In [17]:
df

,title,description,price,area,date,city
0,Parcelle,Détails du produitParcelle size à glo dans la ...,25000000,3600,2024-02,glo
1,Terrains 350m²,Détails du produitBonjour. notre cabinet met e...,12000000,350,2024-02,abomey-calavi
2,Parcelle 325 m2,Détails du produitNous mettons en vente a akpa...,25000000,325,2024-02,akpakpa avotrou
3,Parcelles 631m²,Détails du produitNous mettons en vente pour u...,45000000,684,2024-02,sagla
5,Parcelles,Détails du produitIl est mis en vente à hêvié ...,3200000,500,2024-02,hevié
...,...,...,...,...,...,...
8881,Vente Terrain 450m²,Détails du produitVente d’un ensemble de 10 pa...,2699000,450,2021-02,hevié centre
8882,Vente parcelle 500m²,Détails du produitVente de parcelles de 500m² ...,890000,500,2021-02,zinvié zoumey
8883,Vente parcelles 350 m²,Détails du produitVente d’un ensemble de 22 pa...,399000,350,2021-02,kpomassè
8887,Vente Terrain 501 m²,Détails du produitParcelle de superficie: 501m...,50000000,501,2021-02,cotonou


In [22]:
# Write the DataFrame back to the CSV file
df.to_csv("../terrain_coinafrique_clean.csv", index=False)